In [140]:
import pennylane as qml
import numpy as np
from itertools import permutations
import matplotlib.pyplot as plt


In [136]:
n_comp = 1  # Num of compressed qubit, n_comp<n_inp
n_inp = 2  # Num of input qubits
anc = 1     # Ancilla
totqubit = n_inp+n_comp+anc

allwires = [i for i in range(totqubit)]

dev = qml.device('default.qubit', wires=totqubit)

def ctr_rot(paras,ctr=None,targ=None):
    if ctr is None or targ is None:
            raise ValueError(f"Need to assign wires as control and target.")
    if len(paras) != 3:
            raise ValueError(f"Three Parameters needed; got {len(paras)}.")
    qml.RZ(paras[0], wires=targ)
    qml.CNOT(wires=[ctr,targ])
    qml.RY(paras[1], wires=targ)
    qml.CNOT(wires=[ctr,targ])
    qml.RZ(paras[2], wires=targ)

@qml.qnode(dev)
def circuit(params,inp=None):
    if len(params) != 3*n_inp*(n_inp-1)+6*n_inp:
                raise ValueError(f"Parameters must be a {3*n_inp*(n_inp-1)+6*n_inp} vector; got length {len(params)}.")
    if len(inp) != 2**n_inp:
                raise ValueError(f"Input state-vector length wrong! Got length {len(inp)}")
                
    # Initialization
    qml.AmplitudeEmbedding(features=inp, wires=allwires[:n_inp],normalize=True)
    #qml.AmplitudeEmbedding(features=refs, wires=allwires[n_inp:n_inp+n_comp],normalize=True)
    
    # Prepare U
    paracnt = 0
    for i in range(n_inp):
        qml.RZ(params[paracnt], wires=i)
        qml.RY(params[paracnt+1], wires=i)
        qml.RZ(params[paracnt+2], wires=i)
        paracnt += 3
    
    perm = list(permutations(allwires[:n_inp], 2))
    for i in perm:
        ctr_rot(paras=[params[paracnt],params[paracnt+1],params[paracnt+2]],ctr=i[0],targ=i[1])
        paracnt += 3
    
    for i in range(n_inp):
        qml.RZ(params[paracnt], wires=i)
        qml.RY(params[paracnt+1], wires=i)
        qml.RZ(params[paracnt+2], wires=i)
        paracnt += 3
    
    # SWAP Test
    qml.Hadamard(wires=totqubit-1)
    for i in allwires[n_comp:n_inp]:
        qml.CSWAP(wires=[totqubit-1,i,i+n_comp])
    qml.Hadamard(wires=totqubit-1)
    
    return qml.expval(qml.PauliZ(totqubit-1))

def cost(params):
    C2 = 0
    for inp in inputs:
        res = circuit(params,inp) 
        C2 = C2 + res
    return 1-C2/len(inputs)

In [154]:
init_params = np.random.random(3*n_inp*(n_inp-1)+6*n_inp)

In [157]:
inputs = [[1,1,1,9],[2,1,2,1]]

#opt = qml.GradientDescentOptimizer(stepsize=0.05)
opt = qml.AdagradOptimizer(stepsize=0.1)
# set the number of steps
steps = 1000

params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)
    
    print("Cost after step {:5d}: {:8f}".format(i + 1, cost(params)))
print(f"Optimized params:{params}")

Cost after step     1: 0.543574
Cost after step     2: 0.477166
Cost after step     3: 0.410682
Cost after step     4: 0.355655
Cost after step     5: 0.317366
Cost after step     6: 0.292933
Cost after step     7: 0.277451
Cost after step     8: 0.267453
Cost after step     9: 0.260800
Cost after step    10: 0.256169
Cost after step    11: 0.252753
Cost after step    12: 0.250073
Cost after step    13: 0.247846
Cost after step    14: 0.245906
Cost after step    15: 0.244150
Cost after step    16: 0.242510
Cost after step    17: 0.240943
Cost after step    18: 0.239415
Cost after step    19: 0.237901
Cost after step    20: 0.236380
Cost after step    21: 0.234834
Cost after step    22: 0.233244
Cost after step    23: 0.231594
Cost after step    24: 0.229864
Cost after step    25: 0.228038
Cost after step    26: 0.226095
Cost after step    27: 0.224015
Cost after step    28: 0.221777
Cost after step    29: 0.219360
Cost after step    30: 0.216740
Cost after step    31: 0.213898
Cost aft

Cost after step   262: 0.000010
Cost after step   263: 0.000010
Cost after step   264: 0.000009
Cost after step   265: 0.000009
Cost after step   266: 0.000009
Cost after step   267: 0.000008
Cost after step   268: 0.000008
Cost after step   269: 0.000007
Cost after step   270: 0.000007
Cost after step   271: 0.000007
Cost after step   272: 0.000006
Cost after step   273: 0.000006
Cost after step   274: 0.000006
Cost after step   275: 0.000006
Cost after step   276: 0.000005
Cost after step   277: 0.000005
Cost after step   278: 0.000005
Cost after step   279: 0.000005
Cost after step   280: 0.000004
Cost after step   281: 0.000004
Cost after step   282: 0.000004
Cost after step   283: 0.000004
Cost after step   284: 0.000004
Cost after step   285: 0.000004
Cost after step   286: 0.000003
Cost after step   287: 0.000003
Cost after step   288: 0.000003
Cost after step   289: 0.000003
Cost after step   290: 0.000003
Cost after step   291: 0.000003
Cost after step   292: 0.000003
Cost aft

KeyboardInterrupt: 

In [19]:
for i in range(1,10):
    print(f"n={i}, A: {int(15*i*(i-1)/2)}, \tB: {3*i*(i-1)+6*i}")

n=1, A: 0, 	B: 6
n=2, A: 15, 	B: 18
n=3, A: 45, 	B: 36
n=4, A: 90, 	B: 60
n=5, A: 150, 	B: 90
n=6, A: 225, 	B: 126
n=7, A: 315, 	B: 168
n=8, A: 420, 	B: 216
n=9, A: 540, 	B: 270
